# JAMA revision round 4

In [1]:
!pip install lifelines

     |████████████████████████████████| 348kB 2.8MB/s 


## Import packages



In [2]:
import pandas as pd, numpy as np, re
from numpy import exp, mean
from fancyimpute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from lifelines import CoxPHFitter, AalenJohansenFitter, KaplanMeierFitter
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Read data**

In [0]:
# whole cohort (N=1,478,506)
data_param = "Fortable3mi_ld_dec_censored1.csv" #@param ["colab.csv", "Fortable3mi_ld_dec_censored1.csv"]
d = pd.read_csv('drive/My Drive/facility/'+data_param) 
d['rucc_rural'] = np.where(d.RUCC_2013.isin([1, 2, 3]), 0, (np.where(d.RUCC_2013.isna(), np.nan, 1)))
d = d.drop(['RUCC_2013'], axis=1)

In [35]:
d.shape

(1573056, 47)

In [40]:
d.wlist.value_counts()

0    1351054
1     222002
Name: wlist, dtype: int64

#**Multiple imputation**


##**Random forest imputation for categorical**

In [0]:
for var in ['dialysis_mod1', 'insurance_esrd', 'rucc_rural', 'nephcare_cat2']:
    pred = ['sex_new', 'age_cat', 'race_new', var]
    imputer = IterativeImputer(n_iter=1, random_state=7, predictor=RandomForestClassifier(n_estimators=10))
    imputed = pd.DataFrame(imputer.fit_transform(d[pred]), columns=pred)
    d = d.drop(var, axis=1).join(imputed[var])


##**Bayesian Ridge linear imputation for continuous**


In [0]:
for var in ['Hospitalization_Rate_facility', 'Mortality_Rate_Facility', 'NEAR_DIST']:
    completed = []
    for i in range(5):
        pred = ['sex_new', 'age_cat', 'race_new', var]
        imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i)
        completed.append(imputer.fit_transform(d[pred]))
    completed_mean = np.mean(completed, axis=0)
    imputed = pd.DataFrame(completed_mean, columns=pred)
    if var == 'NEAR_DIST':
        m = imputed[imputed.NEAR_DIST > 0].NEAR_DIST.mean()
        imputed.NEAR_DIST = np.where(imputed.NEAR_DIST < 0, m, imputed.NEAR_DIST)
    d = d.drop(var, axis=1).join(imputed[var])

#**Create cohort for Cox model**

**1) dummy code and order levels based on table**

**2) drop unneeded variables**

In [0]:
# standard cohort
PH_data = d[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',  'profit_txc', 'profit_hosp', #'change_status',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data = PH_data.join(pd.get_dummies(PH_data.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.age_cat, [5, 1, 2, 3, 4, 6], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data = PH_data.join(pd.get_dummies(PH_data.race_new, prefix='race_new', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.chain_class2, [6, 5, 2, 1, 3, 4], True), prefix='chain_class2', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_txc, prefix='profit_txc', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_hosp, prefix='profit_hosp', drop_first=True))
# PH_data = PH_data.join(pd.get_dummies(PH_data.change_status, prefix='change_status', drop_first=True))
PH_data = PH_data.drop(['dialysis_mod1',
            'esrd_cause', 
            'age_cat', 
            'race_new', 
            'chain_class2', 
            # 'change_status',
            'insurance_esrd', 
            'network_us_region_dfr', 
            'profit_txc', 
            'profit_hosp'
            ], axis=1)

# ideal cohort
ideal = d[(d.INC_AGE < 66) &
  (d.pvasc_new == 0) &
  (d.chf == 0) &
  (d.cva_new == 0) &
  (d.PATTXOP_MEDUNFITn == 0)].reset_index(drop=True)

PH_data_ideal = ideal[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.age_cat, [5, 1, 2, 3, 4], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.race_new, prefix='race_new', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data_ideal = PH_data_ideal.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr'], axis=1)

##Table 1

In [0]:
varlist=['chain_class2',
         'for_profit',
          'age_cat',
         'sex_new',
         'race_new',
         'insurance_esrd',
         'esrd_cause',
         'dialysis_mod1',
         'bmi_35',
         'chf',
         'ashd_new',
         'other_cardiac',
         'cva_new',
         'pvasc_new',
         'hypertension',
         'diabetes',
         'copd_new',
         'smoke_new',
         'cancer_new',
         'nephcare_cat2',
         'PATTXOP_MEDUNFITn',
         'network_us_region_dfr',
         'rucc_rural',
         'NEAR_DIST']


In [0]:
with open('test.txt', 'a') as f:
  for groupvar in ['chain_class2', 'for_profit']:
    freq = d.groupby(by=groupvar).esrd_cause.value_counts(sort=False)
    pct = freq.groupby(by=groupvar).apply(lambda x: round(x/x.sum()*100, 1))
    print(pd.concat([freq.rename('count'), pct.rename('pct')], axis=1), file=f)


##**Table 2**

In [46]:
# Truncate follow-up time
def truncate(t):
    e = PH_data.copy()
    if t>0:
      e.loc[e['wl_time'] > t, 'wl']=0
      e.loc[e['wl_time'] > t, 'wl_time']=t
      e.loc[e['ld_time'] > t, 'livingd']=0
      e.loc[e['ld_time'] > t, 'ld_time']=t
      e.loc[e['dec_time'] > t, 'deceasedt']=0
      e.loc[e['dec_time'] > t, 'dec_time']=t
    return e
  

cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in varlist:
    crude = '|'.join([exposure, time, status])
    cph.fit(truncate(0).filter(regex=crude), duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([cph.hazard_ratios_.rename('HR'), exp(cph.confidence_intervals_)], 1), 2))

------------------------------ wl ------------------------------
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.02             0.99             1.04
chain_class2_2  0.91             0.90             0.93
chain_class2_1  0.86             0.85             0.88
chain_class2_3  0.88             0.87             0.90
chain_class2_4  0.82             0.80             0.84
              HR  95% lower-bound  95% upper-bound
for_profit  0.88             0.86             0.89
             HR  95% lower-bound  95% upper-bound
age_cat_1  3.62             3.56             3.68
age_cat_2  2.62             2.58             2.65
age_cat_3  2.12             2.10             2.15
age_cat_4  1.63             1.61             1.65
age_cat_6  0.14             0.14             0.14
           HR  95% lower-bound  95% upper-bound
sex_new  0.74             0.74             0.75
              HR  95% lower-bound  95% upper-bound
race_new_2  1.26             1.25             1.27
race_

In [47]:
for case, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  tmp = d
  count = tmp.groupby('chain_class2')[case].sum().map('{:,}'.format)
  pyr = (tmp.groupby('chain_class2')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count+pyr)
  count1 = tmp.groupby('for_profit')[case].sum().map('{:,}'.format)
  pyr1 = (tmp.groupby('for_profit')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count1+pyr1)

chain_class2
1    73,851 /1,470,874
2    78,880 /1,490,717
3      35,241 /680,501
4      13,727 /285,040
5      10,058 /173,908
6      18,351 /318,129
dtype: object
for_profit
0       28,409 /492,038
1    201,699 /3,927,133
dtype: object
chain_class2
1    13,000 /1,632,923
2    13,194 /1,670,811
3       5,399 /765,230
4       2,256 /316,925
5       2,056 /195,214
6       3,370 /356,370
dtype: object
for_profit
0       5,426 /551,584
1    33,849 /4,385,889
dtype: object
chain_class2
1    29,165 /1,632,923
2    29,574 /1,670,811
3      13,102 /765,230
4       4,805 /316,925
5       4,039 /195,214
6       7,622 /356,370
dtype: object
for_profit
0      11,661 /551,584
1    76,646 /4,385,889
dtype: object



##**Table 3**

In [0]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import statsmodels.formula.api as smf
import statsmodels.api as sm
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
utils = importr('utils')
utils.install_packages('msm')
msm = importr('msm')


In [49]:
# Truncate follow-up time
def truncate(t):
    e = d.copy()
    if t>0:
      e.loc[e['wl_time'] > t, 'wl']=0
      e.loc[e['wl_time'] > t, 'wl_time']=t
      e.loc[e['ld_time'] > t, 'livingd']=0
      e.loc[e['ld_time'] > t, 'ld_time']=t
      e.loc[e['dec_time'] > t, 'deceasedt']=0
      e.loc[e['dec_time'] > t, 'dec_time']=t
    return e

# dataset with aggregate sum for poisson model
for event, time in zip(['wl','livingd', 'deceasedt'],['wl_time', 'ld_time', 'dec_time']):
	for cutoff in [0, 12, 36, 60]:
		poissonset = truncate(cutoff).groupby(['for_profit', 'age_cat', 'sex_new', 'race_new'], as_index=False)[['wl', 'wl_time', 'livingd', 'ld_time', 'dec_time', 'deceasedt']].sum()
		poissonset['age_cat'] = pd.Categorical(poissonset['age_cat'], ordered=True, categories=[5, 1, 2, 3, 4, 6])
		poissonset['race_new'] = pd.Categorical(poissonset['race_new'], ordered=True, categories=[1, 2, 3, 4])

		# poisson regression
		poisson_model = smf.glm(event+'~for_profit+sex_new+age_cat+race_new',
								data=poissonset,
								family=sm.families.Poisson(),
								offset=np.log(poissonset[time]/12)).fit(method='newton')

		# extract coefficient
		b0 = poisson_model.params['Intercept']
		b1 = poisson_model.params['for_profit']

		# calculate rate difference and standard error
		effect = np.exp(b0 + b1) - np.exp(b0)
		vcov = poisson_model.cov_params().loc[['Intercept', 'for_profit'], ['Intercept', 'for_profit']]
		se = msm.deltamethod(ro.Formula('~exp(x1+x2)-exp(x1)'), FloatVector([b0, b1]), ro.r.matrix(vcov.values, nrow=2, ncol=2))
		print("%s %g-months for_profit vs. non_profit: %.2f (%.2f, %.2f) " %(event, cutoff, effect*100, (effect-1.96*float(np.array(se)))*100, (effect+1.96*float(np.array(se)))*100))
    
    


wl 0-months for_profit vs. non_profit: -0.59 (-0.66, -0.52) 
wl 12-months for_profit vs. non_profit: -2.31 (-2.51, -2.11) 
wl 36-months for_profit vs. non_profit: -1.17 (-1.28, -1.05) 
wl 60-months for_profit vs. non_profit: -0.90 (-0.99, -0.81) 
livingd 0-months for_profit vs. non_profit: -0.18 (-0.21, -0.15) 
livingd 12-months for_profit vs. non_profit: -0.39 (-0.46, -0.33) 
livingd 36-months for_profit vs. non_profit: -0.29 (-0.33, -0.24) 
livingd 60-months for_profit vs. non_profit: -0.23 (-0.26, -0.19) 
deceasedt 0-months for_profit vs. non_profit: -0.30 (-0.34, -0.26) 
deceasedt 12-months for_profit vs. non_profit: -0.19 (-0.24, -0.14) 
deceasedt 36-months for_profit vs. non_profit: -0.37 (-0.42, -0.31) 
deceasedt 60-months for_profit vs. non_profit: -0.41 (-0.46, -0.36) 


In [50]:
for event, time in zip(['wl','livingd', 'deceasedt'],['wl_time', 'ld_time', 'dec_time']):
  for cutoff in [0, 12, 36, 60]:
    poissonset = truncate(cutoff).groupby(['chain_class2', 'age_cat', 'sex_new', 'race_new'], as_index=False)[['wl', 'wl_time', 'livingd', 'ld_time', 'dec_time', 'deceasedt']].sum()
    poissonset['chain_class2'] = pd.Categorical(poissonset['chain_class2'], ordered=True, categories=[6, 5, 2, 1, 3, 4])
    poissonset['age_cat'] = pd.Categorical(poissonset['age_cat'], ordered=True, categories=[5, 1, 2, 3, 4, 6])
    poissonset['race_new'] = pd.Categorical(poissonset['race_new'], ordered=True, categories=[1, 2, 3, 4])

    # poisson regression
    poisson_model = smf.glm(event+'~chain_class2+sex_new+age_cat+race_new',
                            data=poissonset,
                            family=sm.families.Poisson(),
                            offset=np.log(poissonset[time]/12)).fit(method='newton')

		# extract coefficient
    for i in [5, 2, 1, 3, 4]:
      b0 = poisson_model.params['Intercept']
      b1 = poisson_model.params['chain_class2[T.'+str(i)+']']
      effect = np.exp(b0 + b1) - np.exp(b0)
      vcov = poisson_model.cov_params().loc[['Intercept', 'chain_class2[T.'+str(i)+']'], ['Intercept', 'chain_class2[T.'+str(i)+']']]
      se = msm.deltamethod(ro.Formula('~exp(x1+x2)-exp(x1)'), FloatVector([b0, b1]), ro.r.matrix(vcov.values, nrow=2, ncol=2))
      print("%s %g-months chain_%g: %.2f (%.2f, %.2f) " %(event, cutoff, i, effect*100, (effect-1.96*float(np.array(se)))*100, (effect+1.96*float(np.array(se)))*100))

wl 0-months chain_5: -0.08 (-0.22, 0.06) 
wl 0-months chain_2: -0.54 (-0.63, -0.44) 
wl 0-months chain_1: -0.66 (-0.75, -0.56) 
wl 0-months chain_3: -0.66 (-0.76, -0.56) 
wl 0-months chain_4: -0.82 (-0.94, -0.71) 
wl 12-months chain_5: 0.05 (-0.34, 0.44) 
wl 12-months chain_2: -2.04 (-2.29, -1.79) 
wl 12-months chain_1: -2.25 (-2.51, -2.00) 
wl 12-months chain_3: -2.71 (-2.98, -2.44) 
wl 12-months chain_4: -2.79 (-3.10, -2.47) 
wl 36-months chain_5: 0.01 (-0.21, 0.23) 
wl 36-months chain_2: -1.04 (-1.18, -0.89) 
wl 36-months chain_1: -1.16 (-1.30, -1.01) 
wl 36-months chain_3: -1.29 (-1.45, -1.14) 
wl 36-months chain_4: -1.51 (-1.69, -1.33) 
wl 60-months chain_5: -0.02 (-0.19, 0.16) 
wl 60-months chain_2: -0.80 (-0.91, -0.69) 
wl 60-months chain_1: -0.92 (-1.04, -0.81) 
wl 60-months chain_3: -0.99 (-1.12, -0.87) 
wl 60-months chain_4: -1.16 (-1.31, -1.02) 
livingd 0-months chain_5: 0.09 (0.03, 0.15) 
livingd 0-months chain_2: -0.14 (-0.18, -0.11) 
livingd 0-months chain_1: -0.13 (-0.16

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains(exposure)].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


------------------------------ wl ------------------------------

 model_0
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.02             0.99             1.04
chain_class2_2  0.91             0.90             0.93
chain_class2_1  0.86             0.85             0.88
chain_class2_3  0.88             0.87             0.90
chain_class2_4  0.82             0.80             0.84

 model_1
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.01             0.98             1.03
chain_class2_2  0.90             0.88             0.91
chain_class2_1  0.87             0.86             0.89
chain_class2_3  0.86             0.85             0.88
chain_class2_4  0.84             0.82             0.86

 model_2
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.01             0.98             1.03
chain_class2_2  0.88             0.86             0.89
chain_class2_1  0.88             0.86             0.89
chain_class2_3  0.85     


##**Figure 2**

In [39]:
d['FIRST_SE'] = pd.to_datetime(d['FIRST_SE'], format='%m/%d/%Y')
fig2 = pd.DataFrame()

for event, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  print('-'*40, event, '-'*40)
  for exposure in ['chain_class2', 'for_profit']:
    print('\n', exposure)
    fig2 = pd.DataFrame()
    for year in range(2001, 2016, 2):
        valid = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')]
        validtime = (pd.to_datetime(str(year+1)+'/12/31') - valid['FIRST_SE']).apply(lambda x: x.days/30.4375)
        valid.loc[valid[time]>validtime, event] = 0
        valid.loc[valid[time]>validtime, time] = validtime
        num = valid.groupby(exposure)[event].sum()
        pyr = valid.groupby(exposure)[time].sum()/12
        rate = (num/pyr*100).rename(str(year)+'-'+"{0:0>2}".format(year+1-2000))
        fig2 = pd.concat([fig2, rate], 1)
    print(fig2.applymap('{:.2f}'.format))

---------------------------------------- wl ----------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    7.49    7.52    8.00    8.15    8.44    8.23    7.75    6.36
2    7.87    7.91    8.85    8.25    8.42    8.42    8.50    6.84
3    7.27    7.78    8.00    7.57    8.72    7.95    7.69    6.54
4    6.63    6.99    7.55    7.93    7.35    7.19    7.08    6.22
5    8.08    9.03   10.74   11.40   10.93   10.61   10.00    8.41
6    9.48    9.63   10.50    9.99   10.29   10.84    9.59    7.93

 for_profit
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
0    9.01    9.43   10.58   10.49   10.51   10.75    9.74    8.09
1    7.54    7.67    8.28    8.07    8.40    8.17    7.97    6.56
---------------------------------------- livingd ----------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    1.59    1.60    1.32    1.17    0.91    0.83    0.7

##Supplemental Table 2

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for i in [6,5,2,1,3,4]:
    print('\n chain_class2={}'.format(i))
    crude = '|'.join(['change_status', status, time])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn'
    if i in [5,6]:
      tmp = PH_data[PH_data.chain_class2==i].filter(regex=model3).drop(['change_status_3'], 1)
    else:
      tmp = PH_data[PH_data.chain_class2==i].filter(regex=model3).drop(['change_status_2'], 1)
    cph.fit(tmp,duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains('change_status_')].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains('change_status_')])], 1), 2))

------------------------------ wl ------------------------------

 chain_class2=6
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.56             0.53     5.900000e-01
change_status_2  0.00             0.00     4.307112e+90

 chain_class2=5
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.40             0.37             0.43
change_status_2  0.01             0.00             0.01

 chain_class2=2
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.02             0.02             0.02
change_status_3  2.64             2.56             2.72

 chain_class2=1
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.04             0.04             0.05
change_status_3  2.84             2.76             2.93

 chain_class2=3
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.01             0.00             0.01
change_status_3  2.11             2.02             2.22

 chain_cl

In [0]:
cph.hazard_ratios_


##**Supplemental Table 3 (ideal cohort)**

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model'+str(i))
      cph.fit(PH_data_ideal.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))



##**Supplemental Table 4**

In [0]:
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for r, region in enumerate(['Northeast', 'South', 'Midwest', 'West']):
    print('\n', region)
    for exposure in ['chain', 'for_profit']:
      crude = '|'.join([exposure, time, status])
      model1 = crude + '|sex_new|age_cat|race_new'
      model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                        'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
      model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #network_us_region_dfr'
      cph.fit(PH_data[d['network_us_region_dfr']==r].filter(regex=model3), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


##Supplemental Table 5 & 6

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['profit_txc', 'profit_hosp']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                      'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn|network_us_region_dfr' #network_us_region_dfr'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.2)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))

## Additional: Cumulative incidence difference

In [37]:
cif = d.copy()
cif.loc[cif['death_wl']==1, 'wl']=2
cif.loc[cif['death_ld']==1, 'livingd']=2
cif.loc[cif['death_dec']==1, 'deceasedt']=2
d_p = cif[cif.for_profit==1]
d_np = cif[cif.for_profit==0]

for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  ajf_p = AalenJohansenFitter(calculate_variance=False).fit(d_p[time]/12, d_p[status], 1)
  ajf_np = AalenJohansenFitter(calculate_variance=False).fit(d_np[time]/12, d_np[status], 1)
  for t in [1, 3, 5, 10]:
    cif_p = ajf_p.cumulative_density_.loc[slice(t)].tail(1).values
    cif_np = ajf_np.cumulative_density_.loc[slice(t)].tail(1).values
    cif_diff = cif_p - cif_np
    se = np.sqrt(cif_p * (1-cif_p) / len(d_p) +cif_np * (1-cif_np) / len(d_np))
    print('%s %g-year profit vs. non-profit: %.1f%% (%.1f%%, %.1f%%)' 
          %(status, t, cif_diff*100, (cif_diff -1.96 *se)*100, (cif_diff+1.96*se )*100))

# for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
#   ajf_p = AalenJohansenFitter(calculate_variance=True).fit(d_p[time]/12, d_p[status], 1)
#   ajf_np = AalenJohansenFitter(calculate_variance=True).fit(d_np[time]/12, d_np[status], 1)
  
#   cif_p = ajf_p.cumulative_density_.loc[slice(10)].tail(1).join(ajf_p.confidence_interval_.loc[slice(10)].tail(1))
#   cif_np = ajf_np.cumulative_density_.loc[slice(10)].tail(1).join(ajf_np.confidence_interval_.loc[slice(10)].tail(1))
#   print(cif_p, cif_np)

wl 1-year profit vs. non-profit: -1.9% (-2.0%, -1.8%)
wl 3-year profit vs. non-profit: -1.9% (-2.1%, -1.8%)
wl 5-year profit vs. non-profit: -2.0% (-2.2%, -1.8%)
wl 10-year profit vs. non-profit: -1.9% (-2.1%, -1.7%)
livingd 1-year profit vs. non-profit: -0.5% (-0.5%, -0.4%)
livingd 3-year profit vs. non-profit: -0.7% (-0.8%, -0.6%)
livingd 5-year profit vs. non-profit: -0.8% (-0.9%, -0.7%)
livingd 10-year profit vs. non-profit: -0.8% (-0.9%, -0.7%)
deceasedt 1-year profit vs. non-profit: -0.2% (-0.2%, -0.2%)
deceasedt 3-year profit vs. non-profit: -0.8% (-0.9%, -0.7%)
deceasedt 5-year profit vs. non-profit: -1.3% (-1.4%, -1.2%)
deceasedt 10-year profit vs. non-profit: -1.3% (-1.5%, -1.2%)


In [38]:
d_c1 = cif[cif.chain_class2==1]
d_c2 = cif[cif.chain_class2==2]
d_c3 = cif[cif.chain_class2==3]
d_c4 = cif[cif.chain_class2==4]
d_c5 = cif[cif.chain_class2==5]
d_c6 = cif[cif.chain_class2==6]


for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  ajf_c1 = AalenJohansenFitter(calculate_variance=False).fit(d_c1[time]/12, d_c1[status], 1)
  ajf_c2 = AalenJohansenFitter(calculate_variance=False).fit(d_c2[time]/12, d_c2[status], 1)
  ajf_c3 = AalenJohansenFitter(calculate_variance=False).fit(d_c3[time]/12, d_c3[status], 1)
  ajf_c4 = AalenJohansenFitter(calculate_variance=False).fit(d_c4[time]/12, d_c4[status], 1)
  ajf_c5 = AalenJohansenFitter(calculate_variance=False).fit(d_c5[time]/12, d_c5[status], 1)
  ajf_c6 = AalenJohansenFitter(calculate_variance=False).fit(d_c6[time]/12, d_c6[status], 1)
  for c, d_len, n in zip([ajf_c5, ajf_c2, ajf_c1, ajf_c3, ajf_c4], [d_c5, d_c2, d_c1, d_c3, d_c4], [5, 2, 1, 3, 4]):
    for t in [1, 3, 5, 10]:
      cif_c6 = ajf_c6.cumulative_density_.loc[slice(t)].tail(1).values
      cif_c = c.cumulative_density_.loc[slice(t)].tail(1).values
      cif_diff = cif_c - cif_c6
      se = np.sqrt(cif_c6 * (1-cif_c6) / len(d_c6) +cif_c * (1-cif_c) / len(d_len))
      print('%s %g-year chain_class %g vs. 6: %.1f%% (%.1f%%, %.1f%%)' 
            %(status, t, n, cif_diff*100, (cif_diff -1.96 *se)*100, (cif_diff+1.96*se )*100))

wl 1-year chain_class 5 vs. 6: 0.0% (-0.2%, 0.3%)
wl 3-year chain_class 5 vs. 6: 0.2% (-0.2%, 0.5%)
wl 5-year chain_class 5 vs. 6: 0.2% (-0.1%, 0.6%)
wl 10-year chain_class 5 vs. 6: 0.5% (0.1%, 0.9%)
wl 1-year chain_class 2 vs. 6: -1.6% (-1.8%, -1.4%)
wl 3-year chain_class 2 vs. 6: -1.4% (-1.6%, -1.2%)
wl 5-year chain_class 2 vs. 6: -1.3% (-1.6%, -1.1%)
wl 10-year chain_class 2 vs. 6: -1.0% (-1.3%, -0.8%)
wl 1-year chain_class 1 vs. 6: -2.0% (-2.2%, -1.8%)
wl 3-year chain_class 1 vs. 6: -2.2% (-2.4%, -1.9%)
wl 5-year chain_class 1 vs. 6: -2.3% (-2.5%, -2.1%)
wl 10-year chain_class 1 vs. 6: -2.2% (-2.4%, -1.9%)
wl 1-year chain_class 3 vs. 6: -2.1% (-2.2%, -1.9%)
wl 3-year chain_class 3 vs. 6: -1.8% (-2.1%, -1.6%)
wl 5-year chain_class 3 vs. 6: -1.8% (-2.1%, -1.6%)
wl 10-year chain_class 3 vs. 6: -1.7% (-1.9%, -1.4%)
wl 1-year chain_class 4 vs. 6: -2.6% (-2.8%, -2.3%)
wl 3-year chain_class 4 vs. 6: -3.0% (-3.3%, -2.7%)
wl 5-year chain_class 4 vs. 6: -3.1% (-3.4%, -2.8%)
wl 10-year chain_